In [1]:

# Set path to STL models in fullXML.py
from mujoco_py import load_model_from_xml, MjSim, MjViewer
import math
import os
import numpy as np
import pandas as pd
import glfw
from fullXML import fullXML
import matplotlib.pyplot as plt
import math

class GetOutOfLoop( Exception ):
    pass

#Convert quaternion from sensor to euler
# Code from https://automaticaddison.com/how-to-convert-a-quaternion-into-euler-angles-in-python/
def euler_from_quaternion(w, x, y, z):
        t0 = +2.0 * (w * x + y * z)
        t1 = +1.0 - 2.0 * (x * x + y * y)
        x_rotation = math.atan2(t0, t1)
     
        t2 = +2.0 * (w * y - z * x)
        t2 = +1.0 if t2 > +1.0 else t2
        t2 = -1.0 if t2 < -1.0 else t2
        y_rotation = math.asin(t2)
     
        t3 = +2.0 * (w * z + x * y)
        t4 = +1.0 - 2.0 * (y * y + z * z)
        z_rotation = math.atan2(t3, t4)
     
        return x_rotation, y_rotation, z_rotation # in radians


class SimulationFull():
    
    delay=1000 #Delay to allow object to move

    #Data columns name
    data_column_names=["motor1", "motor2" ,"motor3" ,"motor4" ,  "x" ,"y", "zRotation",  "round"]


    base_positions=[13,18,22,26,29,33,35,38,41,43,46,48,50,52,54,56,58,60,62,63,65,67,68,70,71,73,74,76,77,78,80,81,82,84,85,86,87]
    digit_positions=[57,70,81,90,99,107,115,122,128,134,141,146,152,157,163,168,173,177,182,187,191,196,200,204,208,212,216,220,224,228,232,236,239,243,247,250,254]

    #Initialise the a sim and viewer model
    def __init__(self,xml,view) -> None:
        #"/home/bci/Dropbox/1 Thesis/code/Simulation/Mujoco/model/AH-underactatedhand_New Sensor.xml"
        self.model = load_model_from_xml(xml)
        self.sim = MjSim(self.model)
        if view:
            self.viewer = MjViewer(self.sim)
            self.viewer._run_speed=64.0
        else:
            self.viewer=None

        self.showsim()

    #Close window
    def close(self):
        if self.viewer is not None :
            glfw.destroy_window(self.viewer.window)

    #Run the simulate setp
    def showsim(self):
        for i in range (self.delay):
            self.sim.step()
            if self.viewer is not None :
                self.viewer.render()


    #Set send control commands
    def ctrl_set_action(self, action):
        """For torque actuators it copies the action into mujoco ctrl field.
        For position actuators it sets the target relative to the current qpos.
        """
        if self.sim.data.ctrl is not None:
            for i in range(action.shape[0]):
                self.sim.data.ctrl[i] = action[i]

    #Set the inital position for the fingers
    def setInitialPosition(self):
        state=self.sim.get_state()
        state.qpos[self.model.get_joint_qpos_addr("fing1")]=0.174533
        state.qpos[self.model.get_joint_qpos_addr("fing2")]=-0.174533
        state.qpos[0:7]= [0,0.100,.04,1,0,0,0]
        self.sim.set_state(state)
        self.ctrl_set_action(np.array([0.174533, 0, -0.174533,0 ]))
        self.sim.forward()
                
        
    def get_sensor_sensordata(self):
        #Data in array [Motor1 , Motor2 ,Motor3 ,Motor4 ,object X ,object Y, object Z ,quat (4 cells w,x,y,z), Leftside Touch, Rightside touch ]
        return self.sim.data.sensordata

    def get_position_sensordata(self):
         return self.sim.data.sensordata[4:7]

    def get_rotation_sensordata(self):
        return self.sim.data.sensordata[7:11]

    def get_motor_data(self):
        return self.sim.data.sensordata[0:4]

    def get_touch_sensordata(self):
        return self.sim.data.sensordata[11:]

    def ConvertRadianToValue(self,value):
        return round(math.degrees(value)/0.29297,0)+512

    
    def ConvertValueToRadian(self,value):
        return math.radians((value-512)*0.29297)

    #This moves the actuators till the object is touched
    def moveToTouch(self):
        control=0
        data=self.get_touch_sensordata()
        
        while np.all(data <= 1000):
            
                control+=0.5
                self.ctrl_set_action(np.array([-control,-control]))
                self.showsim()
                data=self.get_sensor_sensordata()[-2:]
                
            
        return control

    #This moves the actuators till the object is in the start position
    def moveToStart(self):
        #[525, 592, 499, 432]

        radians=[]
        radians.append(self.ConvertValueToRadian(512+(self.base_positions[0]-10)))
        radians.append(self.ConvertValueToRadian(512+self.digit_positions[0]))
        radians.append(self.ConvertValueToRadian(512-(self.base_positions[0]-10)))
        radians.append(self.ConvertValueToRadian(512-self.digit_positions[0]))
        print(radians)
        control=np.array(radians)
        self.ctrl_set_action(control)
        self.showsim()

        return control


      #This moves the object to the new intial position for each round
    def basetocontrol(self,basePositionIndex):
        
        for i in range(basePositionIndex+1):
            #print(i)

            radians=[]
            radians.append(self.ConvertValueToRadian(512+self.base_positions[i]))
            radians.append(self.ConvertValueToRadian(512+self.digit_positions[i]))
            radians.append(self.ConvertValueToRadian(512-self.base_positions[i]))
            radians.append(self.ConvertValueToRadian(512-self.digit_positions[i]))
            control=np.array(radians)
            self.ctrl_set_action(control)
            self.showsim()

        return control

    #Get the data values to be recorded
    def get_data(self,control):
        _,_,zrotation=euler_from_quaternion(*sim.get_rotation_sensordata())

        motorRadians=control.tolist()
        values=[]
        values.append(int(self.ConvertRadianToValue(motorRadians[0])))
        values.append(int(self.ConvertRadianToValue(motorRadians[1])))
        values.append(int(self.ConvertRadianToValue(motorRadians[2])))
        values.append(int(self.ConvertRadianToValue(motorRadians[3])))
        ObjectPostion=self.get_position_sensordata()
        values.extend(ObjectPostion[0:2])
        values.append(zrotation)
        values.append(0)
        return values


    #This moves the actuators till the object is touched
    def movesideways(self,control,direction,steps):

        data=pd.DataFrame(columns=self.data_column_names)
        data.loc[len(data)]=self.get_data(control)
        #print(f'steps {steps} control {control} direction{direction}')

        for i in range(steps):
            count=0
            control[0]+=direction*0.025566 #This is radians equal to increasing motor value by 5
            control[2]+=direction*0.025566 #This is radians equal to increasing motor value by 5

            self.ctrl_set_action(control)

            self.showsim()
            touchvalues=self.get_touch_sensordata()
            #break if the object is lost
            #else record data
            if touchvalues[0] ==0 and touchvalues[1] ==0:
                print("lost touch")
                break
            elif touchvalues[2]>1 or touchvalues[3]>1:
                print("sensors hit")
                break
            else:
                
                data.loc[len(data)]=self.get_data(control)

        return data

    #Run a trajectory
    def runTragectory(self, base, steps, direction):
        done=0
        self.sim.reset() #Reset the sim and set inital position
        sim.setInitialPosition()

        #If the base position is outside the array of calculated positions then at the end
        #break
        if len(self.digit_positions) <=base:
            data=pd.DataFrame(columns=self.data_column_names)
            done=1
            return data,done

        control=self.basetocontrol(base)
    
        data=self.movesideways(control,direction,steps)
        
        data["round"]=base
        return data , done

  

#Defines the arrays for testing multiple values
models=["5R","10R","15R"]
dam=[10,20,30,40,50,60,70,80,90,100]
f_slide=[1,1.1,1.2,1.3,1.4,1.5,1.6,1.7,1.8,1.9,2.0]
f_tor=[0.5,0.05,0.005 ]
f_roll=[0.1,0.01,0.001,0.0001]
solref=[0.5]
sol=[0.95,0.96,0.97,0.98,0.99]

limp1=[[min,max] for min in sol for max in sol if min<=max]


#Cycle through all the arrays
for m in models:
    for damp in dam:
        for f in f_slide:
            for f_t in f_tor:
                for f_r in f_roll:
                    for ref in solref:
                        #make string for solref
                        solr=f'0.001 {ref}'
                        for l in limp1:
                            #Make string for solimp
                            limp=f"{l[0]} {l[1]} 0.0001"

                            #Define the model and start simulator     
                            xml=fullXML(fingertip=m,damping=damp,f_slide=f,f_tor=f_r,f_roll=f_r ,solimp=limp, solref=solr)
                            sim=SimulationFull(xml.xml,0)
                            
                            test=f"{m}_{damp}_{f}_{f_t}_{f_r}_{limp}"
                            print(test)
                            #Create a data frame for data and other variables
                            data=pd.DataFrame(columns=SimulationFull.data_column_names)
                            done=0
                            directions=[-1,1]
                            index=0
                            try:
                                while not done:
                                
                                    #keep runing cycles until done
                                    for d in directions:
                                        data2,done=sim.runTragectory(index,30,d)
                                        
                                        if done:
                                            break
                                        else:
                                            #store data to data frame
                                            data=pd.concat([data,data2])

                                    index+=1
                                    #break if over 75 trials. something has gone wrong
                                    if index>75:
                                        break
                                    #print(index)

                            finally:   
                                #Close the sim and save and plot data
                                sim.close()
                                data.to_csv(f"FullData/full_{damp}_{f}_{f_t}_{f_r}_{limp}.csv")
                                scatter=data.plot.scatter(x='x',y='y')
                                fig = scatter.get_figure()
                                fig.savefig(f"FullData/full_{damp}_{f}_{f_t}_{f_r}_{limp}.png")
                                plt.close(fig)



10R_100_1_0.005_0.0001_0.95 0.96 0.0001
10R_100_1_0.5_0.0001_0.95 0.96 0.0001
10R_100_1_0.05_0.0001_0.95 0.96 0.0001
10R_100_1_0.1_0.0001_0.95 0.96 0.0001
10R_100_1_0.01_0.0001_0.95 0.96 0.0001


In [2]:
sol=[0.95,0.96,0.97,0.98,0.99]

limp1=[[min,max] for min in sol for max in sol if min<=max]
print(limp1)

[[0.95, 0.95], [0.95, 0.96], [0.95, 0.97], [0.95, 0.98], [0.95, 0.99], [0.96, 0.96], [0.96, 0.97], [0.96, 0.98], [0.96, 0.99], [0.97, 0.97], [0.97, 0.98], [0.97, 0.99], [0.98, 0.98], [0.98, 0.99], [0.99, 0.99]]


In [ ]:
from mujoco_py import load_model_from_xml, MjSim, MjViewer
import math
import os
import numpy as np
import pandas as pd
import glfw
from fullXML import fullXML
import matplotlib.pyplot as plt
import math

class GetOutOfLoop( Exception ):
    pass

#Convert quaternion from sensor to euler
def euler_from_quaternion(w, x, y, z):
        t0 = +2.0 * (w * x + y * z)
        t1 = +1.0 - 2.0 * (x * x + y * y)
        x_rotation = math.atan2(t0, t1)
     
        t2 = +2.0 * (w * y - z * x)
        t2 = +1.0 if t2 > +1.0 else t2
        t2 = -1.0 if t2 < -1.0 else t2
        y_rotation = math.asin(t2)
     
        t3 = +2.0 * (w * z + x * y)
        t4 = +1.0 - 2.0 * (y * y + z * z)
        z_rotation = math.atan2(t3, t4)
     
        return x_rotation, y_rotation, z_rotation # in radians


class SimulationFull():
    
    delay=1000 #Delay to allow object to move

    #Data columns name
    data_column_names=["motor1", "motor2" ,"motor3" ,"motor4" ,  "x" ,"y", "zRotation",  "round"]


    base_positions=[13,18,22,26,29,33,35,38,41,43,46,48,50,52,54,56,58,60,62,63,65,67,68,70,71,73,74,76,77,78,80,81,82,84,85,86,87]
    digit_positions=[57,70,81,90,99,107,115,122,128,134,141,146,152,157,163,168,173,177,182,187,191,196,200,204,208,212,216,220,224,228,232,236,239,243,247,250,254]

    #Initialise the a sim and viewer model
    def __init__(self,xml,view) -> None:
        #"/home/bci/Dropbox/1 Thesis/code/Simulation/Mujoco/model/AH-underactatedhand_New Sensor.xml"
        self.model = load_model_from_xml(xml)
        self.sim = MjSim(self.model)
        if view:
            self.viewer = MjViewer(self.sim)
            self.viewer._run_speed=64.0
        else:
            self.viewer=None

        self.showsim()

    #Close window
    def close(self):
        if self.viewer is not None :
            glfw.destroy_window(self.viewer.window)

    #Run the simulate setp
    def showsim(self):
        for i in range (self.delay):
            self.sim.step()
            if self.viewer is not None :
                self.viewer.render()


    #Set send control commands
    def ctrl_set_action(self, action):
        """For torque actuators it copies the action into mujoco ctrl field.
        For position actuators it sets the target relative to the current qpos.
        """
        if self.sim.data.ctrl is not None:
            for i in range(action.shape[0]):
                self.sim.data.ctrl[i] = action[i]

    #Set the inital position for the fingers
    def setInitialPosition(self):
        state=self.sim.get_state()
        state.qpos[self.model.get_joint_qpos_addr("fing1")]=0.174533
        state.qpos[self.model.get_joint_qpos_addr("fing2")]=-0.174533
        state.qpos[0:7]= [0,0.100,.04,1,0,0,0]
        self.sim.set_state(state)
        self.ctrl_set_action(np.array([0.174533, 0, -0.174533,0 ]))
        self.sim.forward()
                
        
    def get_sensor_sensordata(self):
        #Data in array [Motor1 , Motor2 ,Motor3 ,Motor4 ,object X ,object Y, object Z ,quat (4 cells w,x,y,z), Leftside Touch, Rightside touch ]
        return self.sim.data.sensordata

    def get_position_sensordata(self):
         return self.sim.data.sensordata[4:7]

    def get_rotation_sensordata(self):
        return self.sim.data.sensordata[7:11]

    def get_motor_data(self):
        return self.sim.data.sensordata[0:4]

    def get_touch_sensordata(self):
        return self.sim.data.sensordata[11:]

    def ConvertRadianToValue(self,value):
        return round(math.degrees(value)/0.29297,0)+512

    
    def ConvertValueToRadian(self,value):
        return math.radians((value-512)*0.29297)

    #This moves the actuators till the object is touched
    def moveToTouch(self):
        control=0
        data=self.get_touch_sensordata()
        
        while np.all(data <= 1000):
            
                control+=0.5
                self.ctrl_set_action(np.array([-control,-control]))
                self.showsim()
                data=self.get_sensor_sensordata()[-2:]
                
            
        return control

    #This moves the actuators till the object is in the start position
    def moveToStart(self):
        #[525, 592, 499, 432]

        radians=[]
        radians.append(self.ConvertValueToRadian(512+(self.base_positions[0]-10)))
        radians.append(self.ConvertValueToRadian(512+self.digit_positions[0]))
        radians.append(self.ConvertValueToRadian(512-(self.base_positions[0]-10)))
        radians.append(self.ConvertValueToRadian(512-self.digit_positions[0]))
        print(radians)
        control=np.array(radians)
        self.ctrl_set_action(control)
        self.showsim()

        return control


      #This moves the object to the new intial position for each round
    def basetocontrol(self,basePositionIndex):
        
        for i in range(basePositionIndex+1):
            #print(i)

            radians=[]
            radians.append(self.ConvertValueToRadian(512+self.base_positions[i]))
            radians.append(self.ConvertValueToRadian(512+self.digit_positions[i]))
            radians.append(self.ConvertValueToRadian(512-self.base_positions[i]))
            radians.append(self.ConvertValueToRadian(512-self.digit_positions[i]))
            control=np.array(radians)
            self.ctrl_set_action(control)
            self.showsim()

        return control

    #Get the data values to be recorded
    def get_data(self,control):
        _,_,zrotation=euler_from_quaternion(*sim.get_rotation_sensordata())

        motorRadians=control.tolist()
        values=[]
        values.append(int(self.ConvertRadianToValue(motorRadians[0])))
        values.append(int(self.ConvertRadianToValue(motorRadians[1])))
        values.append(int(self.ConvertRadianToValue(motorRadians[2])))
        values.append(int(self.ConvertRadianToValue(motorRadians[3])))
        ObjectPostion=self.get_position_sensordata()
        values.extend(ObjectPostion[0:2])
        values.append(zrotation)
        values.append(0)
        return values


    #This moves the actuators till the object is touched
    def movesideways(self,control,direction,steps):

        data=pd.DataFrame(columns=self.data_column_names)
        data.loc[len(data)]=self.get_data(control)
        #print(f'steps {steps} control {control} direction{direction}')

        for i in range(steps):
            count=0
            control[0]+=direction*0.025566 #This is radians equal to increasing motor value by 5
            control[2]+=direction*0.025566 #This is radians equal to increasing motor value by 5

            self.ctrl_set_action(control)

            self.showsim()
            touchvalues=self.get_touch_sensordata()
            #break if the object is lost
            #else record data
            if touchvalues[0] ==0 and touchvalues[1] ==0:
                print("lost touch")
                break
            elif touchvalues[2]>1 or touchvalues[3]>1:
                print("sensors hit")
                break
            else:
                
                data.loc[len(data)]=self.get_data(control)

        return data

    #Run a trajectory
    def runTragectory(self, base, steps, direction):
        done=0
        self.sim.reset() #Reset the sim and set inital position
        sim.setInitialPosition()

        #If the base position is outside the array of calculated positions then at the end
        #break
        if len(self.digit_positions) <=base:
            data=pd.DataFrame(columns=self.data_column_names)
            done=1
            return data,done

        control=self.basetocontrol(base)
    
        data=self.movesideways(control,direction,steps)
        
        data["round"]=base
        return data , done

  

#Defines the arrays for testing multiple values
models=["5R","10R","15R"]
dam=[10,20,30,40,50,60,70,80,90,100]
f_slide=[1,1.1,1.2,1.3,1.4,1.5,1.6,1.7,1.8,1.9,2.0]
f_tor=[0.5,0.05,0.005 ]
f_roll=[0.1,0.01,0.001,0.0001]
solref=[0.5]
sol=[0.95,0.96,0.97,0.98,0.99]

limp1=[[min,max] for min in sol for max in sol if min<=max]


#Cycle through all the arrays
for m in models:
    for damp in dam:
        for f in f_slide:
            for f_t in f_tor:
                for f_r in f_roll:
                    for ref in solref:
                        #make string for solref
                        solr=f'0.001 {ref}'
                        for l in limp1:
                            #Make string for solimp
                            limp=f"{l[0]} {l[1]} 0.0001"

                            #Define the model and start simulator     
                            xml=fullXML(fingertip=m,damping=damp,f_slide=f,f_tor=f_r,f_roll=f_r ,solimp=limp, solref=solr)
                            sim=SimulationFull(xml.xml,0)
                            
                            test=f"{m}_{damp}_{f}_{f_t}_{f_r}_{limp}"
                            print(test)
                            #Create a data frame for data and other variables
                            data=pd.DataFrame(columns=SimulationFull.data_column_names)
                            done=0
                            directions=[-1,1]
                            index=0
                            try:
                                while not done:
                                
                                    #keep runing cycles until done
                                    for d in directions:
                                        data2,done=sim.runTragectory(index,30,d)
                                        
                                        if done:
                                            break
                                        else:
                                            #store data to data frame
                                            data=pd.concat([data,data2])

                                    index+=1
                                    #break if over 75 trials. something has gone wrong
                                    if index>75:
                                        break
                                    #print(index)

                            finally:   
                                #Close the sim and save and plot data
                                sim.close()
                                data.to_csv(f"FullData/full_{damp}_{f}_{f_t}_{f_r}_{limp}.csv")
                                scatter=data.plot.scatter(x='x',y='y')
                                fig = scatter.get_figure()
                                fig.savefig(f"FullData/full_{damp}_{f}_{f_t}_{f_r}_{limp}.png")
                                plt.close(fig)

